In [2]:
from ift6758.visualizations import AdvancedVisualization

advancedVis = AdvancedVisualization("./../ift6758/data/json_clean/{season}/{season}.pkl")
df = advancedVis.load_season_data(2016)
team = 'Montréal Canadiens'
df_team = advancedVis.get_data_for_team(df,team)
df_team.sample(10)

,team,x,y,shot_count,nb_games,shot_avg,shot_rate_per_hour,shot_rate_diff,shot_rate_diff_percentage,shot_rate_rel_diff
27212,Montréal Canadiens,4.0,-25.0,1,88,0.002278,0.011364,0.009086,79.954442,1.332068
27620,Montréal Canadiens,42.0,-38.0,1,88,0.001519,0.011364,0.009845,86.636295,1.528466
28859,Montréal Canadiens,83.0,-0.0,1,88,0.015945,0.011364,-0.004582,-40.318907,-0.335545
28911,Montréal Canadiens,84.0,21.0,1,88,0.009112,0.011364,0.002252,19.817768,0.219975
28681,Montréal Canadiens,78.0,21.0,1,88,0.012908,0.011364,-0.001544,-13.591496,-0.127266
27437,Montréal Canadiens,35.0,-30.0,1,88,0.018223,0.011364,-0.006860,-60.364465,-0.463692
28873,Montréal Canadiens,83.0,29.0,1,88,0.006834,0.011364,0.004530,39.863326,0.497866
27751,Montréal Canadiens,48.0,3.0,1,88,0.009112,0.011364,0.002252,19.817768,0.219975
28083,Montréal Canadiens,61.0,-17.0,1,88,0.011390,0.011364,-0.000026,-0.227790,-0.002275
28691,Montréal Canadiens,79.0,-23.0,1,88,0.011390,0.011364,-0.000026,-0.227790,-0.002275
